# FreshHarvest Performance Monitoring

This notebook provides comprehensive performance monitoring and analysis for FreshHarvest models.

## Monitoring Overview
- Model inference performance
- Memory usage analysis
- Batch processing benchmarks
- Real-time prediction monitoring
- Resource utilization tracking

In [ ]:
import os
import sys
import time
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import custom modules
from cvProject_FreshHarvest.utils.common import read_yaml, setup_logging

# Setup
setup_logging()
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("FreshHarvest Performance Monitoring Notebook")
print("=" * 55)
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"System: {psutil.cpu_count()} CPUs, {psutil.virtual_memory().total / (1024**3):.1f} GB RAM")

## 1. Model Loading and Setup

In [ ]:
# Load configuration
config = read_yaml('../config/config.yaml')

# Load trained model
model_paths = [
    '../models/trained/best_model.h5',
    '../models/checkpoints/best_model_20250618_100126.h5'
]

model = None
for path in model_paths:
    if os.path.exists(path):
        try:
            model = keras.models.load_model(path)
            print(f"✅ Model loaded from: {path}")
            print(f"Model parameters: {model.count_params():,}")
            break
        except Exception as e:
            print(f"❌ Failed to load {path}: {e}")

if model is None:
    print("⚠️ No model found. Creating lightweight model for demonstration.")
    model = keras.Sequential([
        keras.layers.Input(shape=(224, 224, 3)),
        keras.layers.Conv2D(32, 3, activation='relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(16, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
print(f"\nModel Summary:")
print(f"Input shape: {model.input_shape}")
print(f"Output shape: {model.output_shape}")
print(f"Total parameters: {model.count_params():,}")

## 2. Performance Benchmarking

In [ ]:
def benchmark_inference(model, batch_sizes=[1, 4, 8, 16, 32], num_iterations=10):
    """Benchmark model inference performance."""
    
    results = []
    
    print("🚀 Running inference benchmarks...")
    print("-" * 60)
    
    for batch_size in batch_sizes:
        print(f"Testing batch size: {batch_size}")
        
        # Create dummy data
        dummy_data = np.random.random((batch_size, 224, 224, 3))
        
        # Warm up
        for _ in range(3):
            _ = model.predict(dummy_data, verbose=0)
        
        # Benchmark
        times = []
        memory_usage = []
        
        for i in range(num_iterations):
            # Memory before
            mem_before = psutil.virtual_memory().used / (1024**2)  # MB
            
            # Time inference
            start_time = time.time()
            predictions = model.predict(dummy_data, verbose=0)
            end_time = time.time()
            
            # Memory after
            mem_after = psutil.virtual_memory().used / (1024**2)  # MB
            
            inference_time = end_time - start_time
            times.append(inference_time)
            memory_usage.append(mem_after - mem_before)
        
        # Calculate statistics
        avg_time = np.mean(times)
        std_time = np.std(times)
        avg_memory = np.mean(memory_usage)
        throughput = batch_size / avg_time  # images per second
        
        results.append({
            'batch_size': batch_size,
            'avg_time': avg_time,
            'std_time': std_time,
            'avg_memory': avg_memory,
            'throughput': throughput,
            'time_per_image': avg_time / batch_size
        })
        
        print(f"  Avg time: {avg_time:.4f}s ± {std_time:.4f}s")
        print(f"  Throughput: {throughput:.2f} images/sec")
        print(f"  Memory: {avg_memory:.2f} MB")
        print()
    
    return pd.DataFrame(results)

# Run benchmarks
benchmark_results = benchmark_inference(model)
print("📊 Benchmark Results:")
print(benchmark_results.round(4))

## 3. Performance Visualization

In [ ]:
# Create performance visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Inference time vs batch size
axes[0, 0].plot(benchmark_results['batch_size'], benchmark_results['avg_time'], 'bo-')
axes[0, 0].fill_between(benchmark_results['batch_size'], 
                       benchmark_results['avg_time'] - benchmark_results['std_time'],
                       benchmark_results['avg_time'] + benchmark_results['std_time'],
                       alpha=0.3)
axes[0, 0].set_xlabel('Batch Size')
axes[0, 0].set_ylabel('Inference Time (seconds)')
axes[0, 0].set_title('Inference Time vs Batch Size')
axes[0, 0].grid(True, alpha=0.3)

# 2. Throughput vs batch size
axes[0, 1].plot(benchmark_results['batch_size'], benchmark_results['throughput'], 'go-')
axes[0, 1].set_xlabel('Batch Size')
axes[0, 1].set_ylabel('Throughput (images/sec)')
axes[0, 1].set_title('Throughput vs Batch Size')
axes[0, 1].grid(True, alpha=0.3)

# 3. Time per image vs batch size
axes[1, 0].plot(benchmark_results['batch_size'], benchmark_results['time_per_image'] * 1000, 'ro-')
axes[1, 0].set_xlabel('Batch Size')
axes[1, 0].set_ylabel('Time per Image (ms)')
axes[1, 0].set_title('Time per Image vs Batch Size')
axes[1, 0].grid(True, alpha=0.3)

# 4. Memory usage vs batch size
axes[1, 1].plot(benchmark_results['batch_size'], benchmark_results['avg_memory'], 'mo-')
axes[1, 1].set_xlabel('Batch Size')
axes[1, 1].set_ylabel('Memory Usage (MB)')
axes[1, 1].set_title('Memory Usage vs Batch Size')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance summary
print("\n📈 Performance Summary:")
print(f"Best throughput: {benchmark_results['throughput'].max():.2f} images/sec at batch size {benchmark_results.loc[benchmark_results['throughput'].idxmax(), 'batch_size']}")
print(f"Fastest single image: {benchmark_results['time_per_image'].min()*1000:.2f} ms at batch size {benchmark_results.loc[benchmark_results['time_per_image'].idxmin(), 'batch_size']}")
print(f"Lowest memory usage: {benchmark_results['avg_memory'].min():.2f} MB at batch size {benchmark_results.loc[benchmark_results['avg_memory'].idxmin(), 'batch_size']}")